In [67]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances

Data Loading

In [68]:
ratings_train = pd.read_csv("./Data/rate_train.csv", low_memory=False)
ratings_test = pd.read_csv("./Data/rate_test.csv",low_memory=False)

In [69]:
datasets_train = ratings_train.dataset_id.unique()
model_train = ratings_train.model_id.unique()
datasets_test = ratings_test.dataset_id.unique()
model_test = ratings_test.model_id.unique()
meta_models = pd.read_csv("./Data/models_v.csv",low_memory=False)
models = meta_models.model_id.unique()

Rating Matrix

In [70]:
data_model_train_matrix = pd.DataFrame(index=datasets_train,columns=models)
data_model_test_matrix = pd.DataFrame(index=datasets_test,columns=model_test)

In [71]:
for row in ratings_train.itertuples():
    data_model_train_matrix.loc[row[1]][row[2]] = row[3]


In [72]:
for row in ratings_test.itertuples():
    data_model_test_matrix.loc[row[1]][row[2]] = row[3]
data_model_test_matrix = data_model_test_matrix.fillna(0)

Dataset Similarity Matrix

In [73]:
meta_datasets = pd.read_csv("./Data/dataset_v.csv",low_memory=False)
datasets = meta_datasets.dataset_id.unique()
meta_datasets = meta_datasets.loc[:,("v1","v2","v3","v4","v5","v6","v7","v8","v9","v10","v11","v12","v13","v14","v15","v16")]
scaler = StandardScaler()

# 对dataframe的数据进行标准化
scaled_data = scaler.fit_transform(meta_datasets)
# 将标准化后的数据转换为dataframe，并保留原始索引
scaled_df = pd.DataFrame(scaled_data, index=meta_datasets.index, columns=meta_datasets.columns)
meta_dataset_similarity = cosine_similarity(scaled_df.values.tolist())
meta_dataset_similarity = pd.DataFrame(meta_dataset_similarity,index=datasets,columns=datasets)

In [74]:
meta_dataset_similarity

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,1.000000,0.057282,-0.205998,0.145192,0.020640,0.110559,0.097026,0.276500,-0.351863,-0.047295,...,0.149719,-0.409445,0.263787,0.115303,-0.013954,0.000690,0.002975,0.011482,-0.021570,0.163135
1,0.057282,1.000000,-0.438958,0.578454,0.779882,0.028973,0.540947,-0.554238,0.296891,0.523141,...,-0.539426,0.291139,-0.119987,0.288116,-0.037878,0.072043,0.483767,0.635354,0.361997,0.187997
2,-0.205998,-0.438958,1.000000,-0.210408,-0.600007,0.173321,-0.136983,-0.064347,-0.089264,-0.570692,...,0.383651,-0.332607,-0.106987,-0.151404,-0.029464,0.018348,-0.653393,-0.674113,-0.538035,-0.491216
3,0.145192,0.578454,-0.210408,1.000000,0.625829,0.710368,0.452639,-0.330582,0.063678,0.173019,...,-0.340678,-0.067492,-0.165847,0.162780,-0.026382,0.172264,0.019105,0.389424,0.334551,-0.314283
4,0.020640,0.779882,-0.600007,0.625829,1.000000,0.236301,0.597579,-0.284183,0.279180,0.321405,...,-0.711128,0.226895,-0.218268,0.320114,-0.221069,-0.101544,0.550476,0.686314,0.649043,0.212394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,0.000690,0.072043,0.018348,0.172264,-0.101544,0.063448,0.190238,0.121590,-0.332661,-0.127737,...,0.253976,0.207105,-0.263444,-0.055246,0.326315,1.000000,0.168447,-0.087512,-0.035440,-0.183921
62,0.002975,0.483767,-0.653393,0.019105,0.550476,-0.345970,0.390821,-0.186024,0.003978,0.338140,...,-0.500291,0.411823,-0.031752,0.424013,0.178791,0.168447,1.000000,0.542431,0.506908,0.649412
63,0.011482,0.635354,-0.674113,0.389424,0.686314,-0.064017,0.286442,-0.343642,0.024739,0.696643,...,-0.494205,0.274190,-0.149976,-0.010252,0.115874,-0.087512,0.542431,1.000000,0.828097,0.472056
64,-0.021570,0.361997,-0.538035,0.334551,0.649043,0.118673,0.337409,0.009436,-0.106836,0.386334,...,-0.471695,0.192723,-0.216473,-0.075260,0.214415,-0.035440,0.506908,0.828097,1.000000,0.523058


In [75]:
def cosine_similarity_func(ratings, user1, user2):
    # 找到两个用户共同评分的物品，并将这些评分放入一个向量中
    u1_ratings = ratings.loc[user1].dropna()
    u2_ratings = ratings.loc[user2].dropna()

    common_items = np.intersect1d(u1_ratings.index, u2_ratings.index).tolist()
    u1_common_ratings = u1_ratings.loc[common_items]
    u2_common_ratings = u2_ratings.loc[common_items]

    # 计算两个向量之间的余弦相似度
    if len(common_items) == 0:
        return 0
    else:
        cos_sim = np.dot(u1_common_ratings, u2_common_ratings) / (np.linalg.norm(u1_common_ratings) * np.linalg.norm(u2_common_ratings))
        return cos_sim

In [76]:
dataset_similarity = pd.DataFrame(index=datasets_train,columns=datasets_train)

In [77]:
start_time_train = time.time()

In [78]:
lambda_ = 0.5
for i in datasets_train:
    for j in datasets_train:
        rating_based_sim = cosine_similarity_func(data_model_train_matrix,i,j)
        if rating_based_sim!= 0:
            dataset_similarity.loc[i][j] = rating_based_sim
            continue
        else:
            dataset_similarity.loc[i][j] = 0


In [79]:
dataset_similarity

,0,2,4,5,6,7,8,9,10,11,...,55,56,57,58,59,60,61,62,64,65
0,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1.0,0,0,0,0
61,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.0,0,0,0
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0,0
64,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,0


Prediction

In [80]:
def predict(rating_matrix, similarity_matrix):
    """
    根据评分矩阵和相似度矩阵预测评分。

    参数：
    rating_matrix (pd.DataFrame)：评分矩阵，包含NaN值
    similarity_matrix (numpy.array)：相似度矩阵

    返回：
    pd.DataFrame：预测评分矩阵
    """

    # 获取评分矩阵的均值（忽略NaN值）
    mean_rating = rating_matrix.mean(axis=1).values

    # 将评分矩阵中的NaN值替换为0
    rating_matrix_nan_to_zero = rating_matrix.fillna(0).values

    # 减去均值，得到归一化的评分矩阵
    normalized_rating_matrix = rating_matrix_nan_to_zero - mean_rating[:, np.newaxis]

    # 计算预测评分
    predicted_ratings = mean_rating[:, np.newaxis] + np.dot(similarity_matrix, normalized_rating_matrix) / np.abs(similarity_matrix).sum(axis=1)[:, np.newaxis]

    # 将预测评分数组转换为DataFrame
    predicted_ratings_df = pd.DataFrame(predicted_ratings, index=rating_matrix.index, columns=rating_matrix.columns)

    return predicted_ratings_df

In [81]:
def predict_ratings(rating_matrix, user_similarity_matrix, k=5):
    """
    输入：
    rating_matrix - 评分矩阵，DataFrame格式，其中NaN表示未评分
    user_similarity_matrix - 用户相似度矩阵，DataFrame格式
    k - 最近邻的数量，默认为5

    输出：
    prediction_matrix - 预测矩阵，DataFrame格式
    """

    # 初始化预测矩阵
    prediction_matrix = rating_matrix.copy()

    # 对于评分矩阵中的每个NaN值，使用K最近邻的方法预测评分
    for i in rating_matrix.index:
        for j in rating_matrix.columns:
            if np.isnan(rating_matrix.loc[i][j]):
                # 获取第i个用户的相似度值，并在相似度矩阵中找到K个最相似的用户
                similarity_values = user_similarity_matrix.loc[i].sort_values(ascending=False)[1:k+1]

                # 计算加权平均评分
                weighted_sum = 0
                similarity_sum = 0
                for index, value in similarity_values.items():
                    user_rating = rating_matrix.loc[index][j]
                    if not np.isnan(user_rating):
                        weighted_sum += value * user_rating
                        similarity_sum += value

                # 如果存在至少一个相似用户对该物品进行了评分，则计算预测评分
                if similarity_sum != 0:
                    prediction_matrix.loc[i][j] = weighted_sum / similarity_sum
                else:
                    # 如果没有相似用户评分，则使用当前用户的平均评分作为预测值
                    prediction_matrix.loc[i][j] = rating_matrix.loc[i].mean()

    return prediction_matrix

In [82]:
model_prediction_train = predict_ratings(data_model_train_matrix,dataset_similarity)

In [83]:
model_prediction_train = pd.DataFrame(model_prediction_train,index=datasets_train,columns=models).sort_index().sort_index(axis=1)

In [84]:
model_prediction_test = pd.DataFrame(index=datasets_test,columns=model_test)

In [85]:
end_time_train = time.time()

In [86]:
Training_time = end_time_train - start_time_train
Training_time

26.968526601791382

Metadata Based

In [87]:
def find_sim_index(index):
    row1 = meta_dataset_similarity.loc[index]
    row1_max_index = row1[row1 == row1.max()].index[0]
    return row1_max_index

In [121]:
def Find_Top_k(i,sim_matrix):
    row = sim_matrix.loc[i]
    row = row.sort_values(ascending=False)
    index_row = row.index
    index_row = index_row.values.tolist()
    return index_row

In [122]:
start_time = time.time()

In [123]:
for dataset in datasets_test:
    for model in model_test:
        dataset_sim_list = Find_Top_k(dataset,meta_dataset_similarity)[1:]
        # 仅保留存在于 model_prediction_train 的索引
        valid_indices = [idx for idx in dataset_sim_list if idx in model_prediction_train.index][:15]
        model_prediction_test.loc[dataset][model] = model_prediction_train.loc[valid_indices][model].mean()

In [124]:
valid_indices

[64, 54, 51, 37, 39, 9, 4, 32, 40, 43, 62, 26, 13, 65, 44]

In [125]:
end_time = time.time()

In [126]:
end_time - start_time

0.12468528747558594

In [127]:
for i in datasets_test:
    for j in model_test:
        if data_model_test_matrix.loc[i][j] == 0:
            model_prediction_test.loc[i][j] = None

In [128]:
result = pd.DataFrame(columns={"dataset","model","according_accuracy","groundtruth_according_accuracy"})

In [129]:
for i in datasets_test:
    for j in model_test:
        if model_prediction_test.loc[i][j] is not None:
            according_accuracy = model_prediction_test.loc[i][j]
            groundtruth_according_accuracy = data_model_test_matrix.loc[i][j]
            result = result.append([{'dataset':i,'model':j,'according_accuracy':according_accuracy,'groundtruth_according_accuracy':groundtruth_according_accuracy}],ignore_index=True)

In [130]:
result

,dataset,groundtruth_according_accuracy,model,according_accuracy
0,1,0.949,596,0.834907
1,1,0.947,708,0.834907
2,1,0.946,795,0.834907
3,1,0.925,597,0.834907
4,1,0.914,950,0.846105
5,1,0.905,841,0.834907
6,1,0.901,641,0.834907
7,1,0.883,868,0.837810
8,1,0.851,772,0.833490
9,1,0.508,527,0.845078


In [131]:
result.to_csv("../Huggingface/Output/Rating_only/Full_Rating_only@15.csv",index=False)